In [1]:
%pip install keplergl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pydeck pandas h3 geopandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import geopandas as gpd
from keplergl import KeplerGl

In [4]:
class Disease:
    def __init__(self):
        self.transmission_rate = 0.1
        self.incubation_rate = 0.05
        self.recovery_rate = 0.001
        self.mortality_rate = 0.015

    def seir_rates(self):
        return self.transmission_rate, self.incubation_rate, self.recovery_rate, self.mortality_rate

In [5]:
import random


class PandemicSpreadSimulation:
    def __init__(self):
        gdf = gpd.read_file('kontur_population_20231101_r4.gpkg')

        gdf = gdf[gdf['population'] > 50]

        # Upewnij się, że dane są w odpowiednim układzie współrzędnych (WGS84)
        gdf = gdf.to_crs(epsg=4326)

        # Dodaj pola SEIR do danych
        gdf['S'] = gdf['population']  # Wszyscy początkowo podatni
        gdf['E'] = 0  # Początkowo brak osób narażonych
        gdf['I'] = 0  # Początkowo brak osób zakażonych
        gdf['R'] = 0  # Początkowo brak ozdrowieńców
        gdf['V'] = 0  # Początkowo brak zaszczepionych
        gdf['airport'] = False
        gdf['masks'] = False
        gdf['lockdown'] = 0
        gdf['vaccinations'] = False

        target_hex_id = '846026bffffffff'
        target_hex = gdf[gdf['h3'] == target_hex_id]

        if not target_hex.empty:
            # Przenieś połowę osób z S do I
            half_S = target_hex['S'].values[0] // 2
            gdf.loc[gdf['h3'] == target_hex_id, 'I'] = half_S
            gdf.loc[gdf['h3'] == target_hex_id, 'S'] = target_hex['S'].values[0] - half_S
        else:
            print('No target hex found')

        self.gdf = gdf
        self.disease = Disease()

    def infect_neighbors(self):
        for index, row in self.gdf.iterrows():
            infected = row['I']
            if infected == 0:
                continue

            neighbors = self.gdf[self.gdf.geometry.touches(row.geometry)]

            for neighbor_index, neighbor in neighbors.iterrows():
                neighbor_S = neighbor['S']
                neighbor_population = neighbor['population']

                transmission_probability = 1 - ((1 - infected / row['population']) ** 5)

                is_transmitted = random.random() < transmission_probability
                if is_transmitted:
                    # Przenieś część osób z S do E w sąsiedniej populacji
                    new_exposed = random.randint(1, 10)  # Zakładamy losową liczbę nowych narażonych
                    actual_exposed = min(new_exposed, neighbor_S)  # Nie więcej niż dostępni w S

                    self.gdf.at[neighbor_index, 'S'] -= actual_exposed
                    self.gdf.at[neighbor_index, 'E'] += actual_exposed

        print("Neighbor infections processed.")

    # update simulation state by a day
    def update_simulation(self):
        self.infect_neighbors()

        flights_per_day = 10000
        for _ in range(flights_per_day):
            self.simulate_flight()

        transmission_rate, incubation_rate, recovery_rate, mortality_rate = self.disease.seir_rates()

        for index, row in self.gdf.iterrows():
            S = row['S']
            E = row['E']
            I = row['I']
            R = row['R']
            population = row['population']
            lockdown_days = row['lockdown']

            if lockdown_days > 0:
                self.gdf.at[index, 'lockdown'] = lockdown_days - 1

            # lockdown condition
            if I > population * 0.2 and lockdown_days == 0:
                self.gdf.at[index, 'lockdown'] = 30

            lockdown_transmission_reduce_rate = 0.01
            transmission_rate *= lockdown_transmission_reduce_rate

            new_exposed = int(transmission_rate * S * I / population)
            new_infected = int(incubation_rate * E)
            new_recovered = int(recovery_rate * I)
            new_dead = int(mortality_rate * I)

            # Zaktualizuj wartości SEIR
            new_S = S - new_exposed
            if new_S < 0:
                new_S = 0

            new_E = E + new_exposed - new_infected
            if new_E < 0:
                new_E = 0

            new_I = I + new_infected - new_recovered - new_dead
            if new_I < 0:
                new_I = 0

            new_R = R + new_recovered
            if new_R < 0:
                new_R = 0

            # Uaktualnij dane w gdf
            self.gdf.at[index, 'S'] = new_S
            self.gdf.at[index, 'E'] = new_E
            self.gdf.at[index, 'I'] = new_I
            self.gdf.at[index, 'R'] = new_R

    def simulate_flight(self):
        # Wybierz populacje z lotniskami bez lockdownu
        airports = self.gdf[(self.gdf['airport'] == True) & (self.gdf['lockdown'] == False)]
        if len(airports) < 2:
            return

        origin, destination = airports.sample(2)

        passengers = random.randint(100, 300)

        # Oblicz prawdopodobieństwo, że samolot jest zainfekowany
        infected_probability = 1 - ((1 - origin['E'] / origin['population']) ** passengers)

        # Losuj, czy samolot jest zainfekowany
        is_infected = random.random() < infected_probability

        if is_infected:
            # Jeśli samolot jest zainfekowany, przenieś pasażerów z S do E w docelowej populacji
            destination_S = self.gdf.at[destination.name, 'S']
            new_exposed = min(passengers, destination_S)  # Nie więcej niż S

            self.gdf.at[destination.name, 'S'] -= new_exposed
            self.gdf.at[destination.name, 'E'] += new_exposed

        print(f"Flight from {origin['h3']} to {destination['h3']}")
        print(f"Passengers: {passengers}")
        print(f"Plane infected: {'Yes' if is_infected else 'No'}")

In [6]:
def create_map_with_seir(gdf, filename='kepler_map_seir.html'):
    # Wczytaj dane granic państw
    countries = gpd.read_file("ne_110m_admin_0_countries.shp")  # Granice państw z Natural Earth
    countries.set_crs(epsg=4326, inplace=True)

    # Wczytaj dane miast
    cities = gpd.read_file("ne_110m_populated_places.shp")  # Miasta z Natural Earth
    cities.set_crs(epsg=4326, inplace=True)

    # Zaktualizuj konfigurację mapy
    config = {
        "version": "v1",
        "config": {
            "visState": {
                "filters": [],
                "layers": [
                    # Warstwa H3 Hexagony (jeśli używasz danych H3)
                    {
                        "id": "h3layer",
                        "type": "hexagonId",
                        "config": {
                            "dataId": "population_data",
                            "label": "H3 Hexagons",
                            "color": [0, 255, 0],  # Początkowy kolor zielony
                            "columns": {
                                "hex_id": "h3_index"
                            },
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.8,
                                "colorRange": {
                                    "name": "Custom Gradient",
                                    "type": "sequential",
                                    "category": "Custom",
                                    "colors": ["#00FF00", "#FFA500"]  # Od zielonego do pomarańczowego
                                },
                                "coverage": 1,
                                "enable3d": True,
                                "sizeRange": [0, 500],
                                "coverageRange": [0, 1],
                                "elevationScale": 5
                            },
                        },
                        "visualChannels": {
                            "colorField": {
                                "name": "I",  # Użyj liczby zakażonych jako podstawy koloru
                                "type": "real"
                            },
                            "colorScale": "quantile",
                            "sizeField": {
                                "name": "population",
                                "type": "real"
                            },
                            "sizeScale": "linear"
                        }
                    },
                    # Warstwa GeoJSON (jeśli chcesz wyświetlać granice państw w formacie GeoJSON)
                    {
                        "id": "geojsonlayer",
                        "type": "geojson",
                        "config": {
                            "dataId": "countries",
                            "label": "Countries",
                            "color": [200, 200, 200],  # Ustaw kolor konturów na szary
                            "columns": {
                                "geojson": "geometry"
                            },
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.8,
                                "colorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": ["#D3D3D3", "#D3D3D3"]  # Jasno szary
                                },
                                "coverage": 1
                            }
                        }
                    },
                    # Warstwa heksagonów z danymi o zarażonych
                    {
                        "id": "infected_layer",
                        "type": "hexagonId",
                        "config": {
                            "dataId": "population_data",
                            "label": "Infected Population",
                            "columns": {
                                "hex_id": "h3"
                            },
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.8,
                                "colorRange": {
                                    "name": "Infection Scale",
                                    "type": "quantize",
                                    "category": "Custom",
                                    "colors": [
                                        "#FFFF00",  # 0-132.7k
                                        "#FFB000",  # 132.7k-265.4k
                                        "#FF6000",  # 265.4k-398.1k
                                        "#FF0000"   # 398.1k-530.9k
                                    ],
                                    "steps": 4
                                },
                                "coverage": 1,
                                "enable3d": True,
                                "sizeRange": [0, 500],
                                "coverageRange": [0, 1],
                                "elevationScale": 5,
                                "heightRange": [0, 500],
                                "colorScaleType": "quantize",
                                "strokeColor": [255, 255, 255],
                                "strokeColorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": ["#FFFFFF"]
                                },
                                "domains": [0, 132725, 265450, 398175, 530900]  # Przedziały dla skali
                            }
                        },
                        "visualChannels": {
                            "colorField": {
                                "name": "I",
                                "type": "real",
                                "scale": "quantize",
                                "domain": [0, 530900]  # Min i max wartości
                            },
                            "colorScale": "quantize",
                            "heightField": {
                                "name": "I",
                                "type": "real",
                                "scale": "linear",
                                "domain": [0, 530900]  # Min i max wartości
                            },
                            "sizeField": {
                                "name": "population",
                                "type": "real"
                            },
                            "sizeScale": "linear",
                            "heightScale": "linear"
                        }
                    },
                    # Warstwa Miast
                    {
                        "id": "cityLayer",
                        "type": "geojson",
                        "config": {
                            "dataId": "cities",
                            "label": "Cities",
                            "color": [255, 0, 0],  # Kolor czerwony dla miast
                            "columns": {
                                "geojson": "geometry"
                            },
                            "isVisible": True
                        }
                    }
                ]
            },
            "mapState": {
                "bearing": 0,
                "latitude": 0,
                "longitude": 0,
                "pitch": 45,
                "zoom": 2,
                "dragRotate": True
            },
            "mapStyle": {
                "styleType": "dark",
                "topLayerGroups": {},
                "visibleLayerGroups": {
                    "label": True,
                    "road": True,
                    "border": True,  # Włącz granice
                    "building": True,
                    "water": True,
                    "land": True
                }
            }
        }
    }

    # Stwórz mapę z określoną wysokością
    map_1 = KeplerGl(height=800)

    # Dodaj dane do mapy
    map_1.add_data(data=gdf, name='population_data')
    map_1.add_data(data=countries, name="Countries")
    map_1.add_data(data=cities, name="Cities")

    # Zastosuj konfigurację
    map_1.config = config

    # Zapisz do pliku HTML
    map_1.save_to_html(file_name=filename)

    


In [7]:
simulation = PandemicSpreadSimulation()
# create_map_with_seir(simulation.gdf, 'day0.html')
# for i in range(2):
#     simulation.update_simulation()
#     create_map_with_seir(simulation.gdf, f'day{i + 1}.html')


In [9]:
simulation
simulation_states = []
initial_state = simulation.gdf.copy()
simulation_states.append(initial_state)

num_days = 10

# Wykonaj symulację dla pozostałych dni
print(f"\nRozpoczynam symulację dla {num_days-1} kolejnych dni...")
for day in range(num_days - 1):
    print(f"\nDzień {day + 1}:")
    simulation.update_simulation()
    current_state = simulation.gdf.copy()
    print(f"- Liczba zarażonych (suma I): {current_state['I'].sum()}")
    print(f"- Liczba podatnych (suma S): {current_state['S'].sum()}")
    simulation_states.append(current_state)
print("koniec")


Rozpoczynam symulację dla 9 kolejnych dni...

Dzień 1:
Neighbor infections processed.
- Liczba zarażonych (suma I): 505817
- Liczba podatnych (suma S): 8031246939.0

Dzień 2:
Neighbor infections processed.
- Liczba zarażonych (suma I): 497728
- Liczba podatnych (suma S): 8031246927.0

Dzień 3:
Neighbor infections processed.
- Liczba zarażonych (suma I): 489769
- Liczba podatnych (suma S): 8031246918.0

Dzień 4:
Neighbor infections processed.
- Liczba zarażonych (suma I): 481937
- Liczba podatnych (suma S): 8031246906.0

Dzień 5:
Neighbor infections processed.
- Liczba zarażonych (suma I): 474232
- Liczba podatnych (suma S): 8031246893.0

Dzień 6:
Neighbor infections processed.
- Liczba zarażonych (suma I): 466650
- Liczba podatnych (suma S): 8031246876.0

Dzień 7:
Neighbor infections processed.
- Liczba zarażonych (suma I): 459191
- Liczba podatnych (suma S): 8031246860.0

Dzień 8:
Neighbor infections processed.
- Liczba zarażonych (suma I): 451851
- Liczba podatnych (suma S): 8031246

In [12]:
def create_map_with_simulation(simulation_states, filename='pandemic_simulation.html'):
    """
    Tworzy mapę z warstwami dla każdego dnia symulacji
    """
    print("Rozpoczynam tworzenie mapy...")
    
    try:
        # Wczytaj dane granic państw i miast
        print("Wczytuję dane granic państw...")
        countries = gpd.read_file("ne_110m_admin_0_countries.shp")
        countries.set_crs(epsg=4326, inplace=True)
        print("Wczytuję dane miast...")
        cities = gpd.read_file("ne_110m_populated_places.shp")
        cities.set_crs(epsg=4326, inplace=True)
        print("Dane bazowe wczytane pomyślnie")

        # Lista do przechowywania wszystkich stanów symulacji
        # simulation_states = []
        
        # Zapisz stan początkowy
        # print("Zapisuję stan początkowy symulacji...")
        initial_state = simulation_states[0]
        print(f"Stan początkowy - liczba rekordów: {len(initial_state)}")
        print(f"Kolumny w danych: {initial_state.columns.tolist()}")
        # simulation_states.append(initial_state)
        


        # Stwórz mapę
        print("\nTworzę obiekt mapy...")
        map_1 = KeplerGl(height=800)
        
        # Dodaj podstawowe dane
        print("Dodaję dane bazowe do mapy...")
        map_1.add_data(data=countries, name="countries")
        map_1.add_data(data=cities, name="cities")
        
        # Dodaj dane dla każdego dnia
        print("\nDodaję dane dla poszczególnych dni...")
        for day, state in enumerate(simulation_states):
            print(f"Dodaję dzień {day}...")
            print(f"- Kształt danych: {state.shape}")
            print(f"- Kolumna h3 obecna: {'h3' in state.columns}")
            print(f"- Kolumna I obecna: {'I' in state.columns}")
            map_1.add_data(data=state, name=f'pandemic_day_{day}')

        print("\nPrzygotowuję konfigurację warstw...")
        # Przygotuj warstwy dla każdego dnia
        layers = [
            # Warstwa granic państw
            {
                "id": "country_borders",
                "type": "geojson",
                "config": {
                    "dataId": "countries",
                    "label": "Countries",
                    "color": [200, 200, 200],
                    "columns": {"geojson": "geometry"},
                    "isVisible": True,
                    "visConfig": {
                        "opacity": 0.3,
                        "thickness": 1
                    }
                }
            },
            # Warstwa miast
            {
                "id": "cities_layer",
                "type": "geojson",
                "config": {
                    "dataId": "cities",
                    "label": "Cities",
                    "color": [255, 0, 0],
                    "columns": {"geojson": "geometry"},
                    "isVisible": True
                }
            }
        ]

        # Dodaj warstwę dla każdego dnia symulacji
        for day in range(len(simulation_states)):
            print(f"Dodaję konfigurację dla dnia {day}...")
            layer = {
                "id": f"infected_layer_day_{day}",
                "type": "hexagonId",
                "config": {
                    "dataId": f"pandemic_day_{day}",
                    "label": f"Infected Population - Day {day}",
                    "columns": {
                        "hex_id": "h3"
                    },
                    "isVisible": day == 0,
                    "visConfig": {
                        "opacity": 0.8,
                        "colorRange": {
                            "name": "Infection Scale",
                            "type": "threshold",
                            "category": "Custom",
                            "colors": [
                                "#80ff00",  # 0 zarażonych
                                "#FFFF00",  # 1-128.5k
                                "#FFB000",  # 128.5k-257k
                                "#FF6000",  # 257k-385.5k
                                "#FF0000"   # 385.5k-514k
                            ]
                        },
                        "coverage": 1,
                        "enable3d": True,
                        "sizeRange": [0, 500],
                        "coverageRange": [0, 1],
                        "elevationScale": 5,
                        "heightRange": [0, 500],
                        "colorScaleType": "threshold"
                    }
                },
                "visualChannels": {
                    "colorField": {
                        "name": "I",
                        "type": "real",
                        "scale": "threshold",
                        "domain": [0, 1, 128500, 257000, 385500, 514000]
                    },
                    "heightField": {
                        "name": "I",
                        "type": "real",
                        "scale": "linear",
                        "domain": [0, 514000]
                    },
                    "sizeField": {
                        "name": "population",
                        "type": "real"
                    },
                    "sizeScale": "linear",
                    "heightScale": "linear"
                }
            }
            
            layers.append(layer)

        # Konfiguracja mapy
        print("Tworzę końcową konfigurację mapy...")
        config = {
            "version": "v1",
            "config": {
                "visState": {
                    "filters": [],
                    "layers": layers
                },
                "mapState": {
                    "bearing": 0,
                    "latitude": 50,
                    "longitude": 20,
                    "pitch": 45,
                    "zoom": 4,
                    "dragRotate": True
                },
                "mapStyle": {
                    "styleType": "dark",
                    "topLayerGroups": {},
                    "visibleLayerGroups": {
                        "label": True,
                        "road": True,
                        "border": True,
                        "building": True,
                        "water": True,
                        "land": True
                    }
                }
            }
        }

        print("\nStosuje konfigurację do mapy...")
        map_1.config = config

        print(f"\nZapisuję mapę do pliku {filename}...")
        map_1.save_to_html(file_name=filename)
        
        print("\nMapa została utworzona pomyślnie!")
        return map_1

    except Exception as e:
        print(f"\nWYSTĄPIŁ BŁĄD: {str(e)}")
        import traceback
        print("\nSzczegóły błędu:")
        print(traceback.format_exc())
        raise

# Przykład użycia:
try:
    print("\nUruchamiam tworzenie mapy z symulacją...")
    map_1 = create_map_with_simulation(simulation_states)
    print("\nGotowe! Wyświetlam mapę...")
    map_1
except Exception as e:
    print(f"\nBłąd podczas wykonywania głównego kodu: {str(e)}")


Uruchamiam tworzenie mapy z symulacją...
Rozpoczynam tworzenie mapy...
Wczytuję dane granic państw...
Wczytuję dane miast...
Dane bazowe wczytane pomyślnie
Stan początkowy - liczba rekordów: 60790
Kolumny w danych: ['h3', 'population', 'geometry', 'S', 'E', 'I', 'R', 'V', 'airport', 'masks', 'lockdown', 'vaccinations']

Tworzę obiekt mapy...
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Dodaję dane bazowe do mapy...

Dodaję dane dla poszczególnych dni...
Dodaję dzień 0...
- Kształt danych: (60790, 12)
- Kolumna h3 obecna: True
- Kolumna I obecna: True
Dodaję dzień 1...
- Kształt danych: (60790, 12)
- Kolumna h3 obecna: True
- Kolumna I obecna: True
Dodaję dzień 2...
- Kształt danych: (60790, 12)
- Kolumna h3 obecna: True
- Kolumna I obecna: True
Dodaję dzień 3...
- Kształt danych: (60790, 12)
- Kolumna h3 obecna: True
- Kolumna I obecna: True
Dodaję dzień 4...
- Kształt danych: (60790, 12)
- Kolumna h3 obecna: True
- Kolumna I obecna: True
Dodaję dzień 5...
- Kształt danych